In [1]:
%load_ext lab_black

In [2]:
from shading_model_ui import ShadingModelInputUi

In [3]:
import pathlib
from ipyautoui import AutoUi, AutoVjsf, AutoDisplay, demo
from pydantic import BaseModel, Field
from ipyautoui.custom.filechooser import FileChooser

In [10]:
# demo()

In [4]:
ui = ShadingModelInputUi()
display(ui)

ShadingModelInputUi(children=(AutoObject(children=(VBox(children=(AutoBox(children=(HBox(children=(Checkbox(va…

In [16]:
ui.value

{'run_irradiance_calc': True,
 'run_df_calc': True,
 'run_climate_based_dl_model': False,
 'epw_filepath': 'J:\\J7356\\Calcs\\DaylightOverheatingTool\\02_InputData\\00_Defaults\\00_WeatherData\\London_LHR_DSY1_2020High50.epw',
 'bearing': 0.0,
 'start_month': 1,
 'start_day': 1,
 'start_hour': 0,
 'end_month': 12,
 'end_day': 31,
 'end_hour': 23,
 'timestep': 1,
 'window_grid_offset_distance': -0.05,
 'window_grid_size': 0.1,
 'working_plane_grid_size': 0.1,
 'working_plane_height': 0.8,
 'calc_surface_offset': 0.0,
 'height_above_ground_level': 0.0,
 'x_offset': 0.0,
 'y_offset': 0.0,
 'room_name': 'parametric_room',
 'room_width': 3.0,
 'room_depth': 4.0,
 'room_height': 3.0,
 'wall_thickness': 0.3,
 'slab_thickness': 0.4,
 'internal_floor_reflectance': 0.2,
 'internal_wall_reflectance': 0.5,
 'internal_ceiling_reflectance': 0.7,
 'external_wall_reflectance': 0.2,
 'external_roof_reflectance': 0.2,
 'external_soffit_reflectance': 0.2,
 'reveal_reflectance': 0.4,
 'glazing_u_value': 0

In [14]:
def test_filechooser():
    class Test(BaseModel):
        path: pathlib.Path = Field(
            ".", json_schema_extra=dict(filter_pattern=["*_.py"])
        )  # note. filter_pattern ipyfilechooser kwarg passed on
        string: str = "test"

    ui = AutoUi(Test)
    display(ui)
    print(ui.value)
    assert isinstance(ui.di_widgets["path"], FileChooser)
    assert ui.di_widgets["path"].value == "."
    # assert ui.di_widgets["path"].filter_pattern == ["*_.py"]

In [4]:
import seedir
p = pathlib
seedir.seedir("template_study")

NameError: name 'pathlib' is not defined

In [24]:
import uuid
uuid.UUID()

TypeError: one of the hex, bytes, bytes_le, fields, or int arguments must be given

In [3]:
ui.value

{'run_irradiance_calc': True,
 'run_df_calc': True,
 'run_climate_based_dl_model': False,
 'epw_filepath': '.',
 'bearing': 0.0,
 'start_month': 1,
 'start_day': 1,
 'start_hour': 0,
 'end_month': 12,
 'end_day': 31,
 'end_hour': 23,
 'timestep': 1,
 'irr_grid_offset_distance': -0.05,
 'irr_grid_size': 0.1,
 'df_grid_size': 0.1,
 'working_plane_height': 0.8,
 'calc_surface_offset': 0.0,
 'height_above_ground_level': 0.0,
 'room_name': 'parametric_room',
 'room_width': 3.0,
 'room_depth': 4.0,
 'room_height': 3.0,
 'wall_thickness': 0.3,
 'slab_thickness': 0.4,
 'glass_u_value': 0.8,
 'g_value': 0.4,
 'glass_vlt': 0.7,
 'frame_thickness': 0.05,
 'toggle_window': True,
 'window_offset': 0.0,
 'window_width': 1.0,
 'sill_height': 1.1,
 'window_height': 1.5,
 'extra_window_reveal_depth': 0.0,
 'toggle_door': False,
 'door_offset': 0.0,
 'door_width': 1.0,
 'threshold_height': 0.0,
 'door_height': 2.4,
 'toggle_overhang': False,
 'overhang_width': 1.0,
 'overhang_depth': 1.0,
 'overhang_hei

In [9]:
def save_json(json_data, folder_path, name, overwrite=False):
    """
    Appends filepath and saves the data to the folder. Creates a random filename if none is specified (wait this bit doesn't actually work, will fix later)
    TODO - what do I need to return here
    """
    if not name:
        name = str(uuid.uuid4())
    filename = str(name) + ".json"
    filepath = os.path.join(folder_path, filename)
    if not overwrite:
        if os.path.exists(filepath):
            name = name + "_" + str(uuid.uuid4())
            filename = name + ".json"
            filepath = os.path.join(folder_path, filename)
    results_filename = str(name) + "_results.json"
    results_filepath = os.path.join(folder_path, results_filename)
    fp_dict = {
        "data_filepath": filepath,
        "folder_path": folder_path,
        "filename": filename,
        "results_filename": results_filename,
    }
    data = fp_dict | json_data  # Appends filepath information to start of dict

    with open(filepath, "w") as outfile:
        json.dump(data, outfile)
    return filepath, filename

In [10]:
def parameter_variation(
    parent_folder,
    parameter_name,
    start_value,
    end_value,
    step,
    global_params={},
    overwrite=False,
    timestamped=True,
):
    """
    Varies a desired parameter through a specific range
    Creates a folder of json files that store the required run parameters

    """
    param_folder_path = os.path.join(parent_folder, parameter_name)
    pathlib.Path(param_folder_path).mkdir(parents=True, exist_ok=True)
    if timestamped:
        timestamp = datetime.datetime.now().strftime("%Y%m%d_%H%M%S")
        timestamped_folder_path = os.path.join(param_folder_path, timestamp)
        pathlib.Path(timestamped_folder_path).mkdir(parents=True, exist_ok=True)
        param_folder_path = timestamped_folder_path
    ui = ShadingModelInputUi()
    if global_params:
        ui.value = global_params  # Set the globalparameters (if specified - this won't work with the deafult!!!)
    param_array = np.linspace(
        start_value, end_value, int(round((end_value - start_value) / step, 0) + 1)
    ).round(decimals=3)
    names_list = []
    filename_list = []
    filepath_list = []
    for i, param_val in enumerate(param_array):
        val_full_name = parameter_name + "_" + str(param_val)
        param_val_folder_path = os.path.join(param_folder_path, val_full_name)
        pathlib.Path(param_val_folder_path).mkdir(parents=True, exist_ok=True)

        ui.value = {parameter_name: param_val}
        fp, fn = save_json(ui.value, param_val_folder_path, val_full_name, overwrite)
        names_list.append(val_full_name)
        filepath_list.append(fp)
        filename_list.append(fn)
    return names_list, filename_list, filepath_list

In [11]:
def parameter_file_creation(
    parent_folder, type_name, subtype_name, params, overwrite=False, timestamped=True
):
    """
    TODO - should params be an optional parameter? How to force correct variable type usage
    """
    type_folder_path = os.path.join(parent_folder, type_name)
    pathlib.Path(type_folder_path).mkdir(parents=True, exist_ok=True)
    if timestamped:
        timestamp = datetime.datetime.now().strftime("%Y%m%d_%H%M%S")
        timestamped_folder_path = os.path.join(type_folder_path, timestamp)
        pathlib.Path(timestamped_folder_path).mkdir(parents=True, exist_ok=True)
        type_folder_path = timestamped_folder_path
    ui = ShadingModelInputUi()
    ui.value = params
    subtype_folder_path = os.path.join(type_folder_path, subtype_name)
    pathlib.Path(subtype_folder_path).mkdir(parents=True, exist_ok=True)
    fp, fn = save_json(ui.value, subtype_folder_path, subtype_name, overwrite)
    return fp, fn

In [12]:
def fixed_timestamp_parameter_file_creation(
    parent_folder, type_name, subtype_name, params, timestamp, overwrite=False
):
    """
    This should allow multiple file creations under the same timestamp
    """
    type_folder_path = os.path.join(parent_folder, type_name)
    pathlib.Path(type_folder_path).mkdir(parents=True, exist_ok=True)
    # timestamp = datetime.datetime.now().strftime("%Y%m%d_%H%M%S")
    timestamped_folder_path = os.path.join(type_folder_path, timestamp)
    pathlib.Path(timestamped_folder_path).mkdir(parents=True, exist_ok=True)
    type_folder_path = timestamped_folder_path
    ui = ShadingModelInputUi()
    ui.value = params
    subtype_folder_path = os.path.join(type_folder_path, subtype_name)
    pathlib.Path(subtype_folder_path).mkdir(parents=True, exist_ok=True)
    fp, fn = save_json(ui.value, subtype_folder_path, subtype_name, overwrite)
    return fp, fn

## File Generation

In [13]:
LINUX_FDIR_JOBS = pathlib.PurePosixPath("/home/jovyan/jobs/")
LINUX_FDIR_RHINO_RESULTS = LINUX_FDIR_JOBS / "J7356/Calcs/Modelling/Rhino/Results"
EPW_FPATH = "/home/jovyan/jobs/J7356/Calcs/Modelling/Rhino/WeatherFiles/London_LHR_DSY1_2020High50.epw"

In [14]:
type_name = "horizontal_slats"
subtype_name = "optimised_dimensions"

parameters = {
    "run_irradiance_calc": True,
    "run_df_calc": True,
    "run_climate_based_dl_model": False,
    "epw_filepath": EPW_FPATH,
    "bearing": 180,
    "start_month": 9,
    "start_day": 20,
    "start_hour": 0,
    "end_month": 9,
    "end_day": 20,
    "end_hour": 23,
    "timestep": 1,
    "irr_grid_offset_distance": -0.05,
    "irr_grid_size": 0.1,
    "df_grid_size": 0.1,
    "working_plane_height": 0.8,
    "calc_surface_offset": 0,
    "height_above_ground_level": 0.0,
    "room_width": 3.0,
    "room_depth": 4.0,
    "room_height": 3.0,
    "wall_thickness": 0,
    "slab_thickness": 0,
    "glass_u_value": 0.8,
    "g_value": 0.4,
    "glass_vlt": 0.7,
    "frame_thickness": 0,
    "toggle_window": True,
    "window_offset": 0.0,
    "window_width": 1.0,
    "sill_height": 1.1,
    "window_height": 1.5,
    "extra_window_reveal_depth": 0.0,
    "toggle_door": False,
    "door_offset": 0.0,
    "door_width": 1.0,
    "threshold_height": 0.0,
    "door_height": 2.4,
    "toggle_overhang": False,
    "overhang_width": 1.4,
    "overhang_depth": 1.03,
    "overhang_height": 2.6,
    "overhang_offset": 0.0,
    "overhang_reflectance": 0.5,
    "overhang_angle": 14.04,
    "toggle_louvres_or_fins": True,
    "fins_or_louvers": False,
    "louvre_blade_depth": 0.2,
    "number_louvre_shades": 10,
    "distance_between_louvre_shades": 0.2,
    "louvre_blade_angle": 0.0,
    "louvre_reflectance": 0.35,
    "toggle_blind": False,
    "blind_percentage_cover": 0.0,
    "blind_offset": 0.0,
    "blind_solar_reflectance": 0.5,
    "blind_visible_reflectance": 0.5,
    "blind_diffusive_transmittance": 0.2,
    "blind_specular_transmittance": 0.04,
    "toggle_opaque_blind": False,
    "toggle_perforated_blind": False,
    "blind_perforation_spacing": 0.04,
    "blind_perforation_radius": 0.01,
    "toggle_horizontal_blind": False,
    "toggle_reverse_blind_direction": False,
    "toggle_custom_shading": False,
    "custom_shade_filepath": ".",
    "toggle_external_vertical_fin": False,
    "vertical_fin_height": 2.3,
    "vertical_fin_depth": 0.6,
    "fin_horizontal_offset": 0.5,
    "fin_vertical_offset": 0.7,
    "vertical_fin_reflectance": 0.35,
}

filepath, filename = parameter_file_creation(
    LINUX_FDIR_RHINO_RESULTS, type_name, subtype_name, parameters
)
print(filepath)

Timestep. multiple matches found. using the last one.
['Dropdown', 'IntText']
/home/jovyan/jobs/J7356/Calcs/Modelling/Rhino/Results/horizontal_slats/20231023_102226/optimised_dimensions/optimised_dimensions.json


In [14]:
type_name = "horizontal_slats"
subtype_name = "external_venetian_blind_rotated"

parameters = {
    "run_irradiance_calc": True,
    "run_df_calc": False,
    "run_climate_based_dl_model": True,
    "epw_filepath": EPW_FPATH,
    "bearing": 270,
    "start_month": 9,
    "start_day": 20,
    "start_hour": 0,
    "end_month": 9,
    "end_day": 20,
    "end_hour": 23,
    "timestep": 1,
    "irr_grid_offset_distance": -0.05,
    "irr_grid_size": 0.1,
    "df_grid_size": 0.1,
    "working_plane_height": 0.8,
    "calc_surface_offset": 0,
    "height_above_ground_level": 0.0,
    "room_width": 3.0,
    "room_depth": 4.0,
    "room_height": 3.0,
    "wall_thickness": 0,
    "slab_thickness": 0,
    "glass_u_value": 0.8,
    "g_value": 0.4,
    "glass_vlt": 0.7,
    "frame_thickness": 0,
    "toggle_window": True,
    "window_offset": 0.0,
    "window_width": 1.0,
    "sill_height": 1.1,
    "window_height": 1.5,
    "extra_window_reveal_depth": 0.0,
    "toggle_door": False,
    "door_offset": 0.0,
    "door_width": 1.0,
    "threshold_height": 0.0,
    "door_height": 2.4,
    "toggle_overhang": False,
    "overhang_width": 1.4,
    "overhang_depth": 1.03,
    "overhang_height": 2.6,
    "overhang_offset": 0.0,
    "overhang_reflectance": 0.5,
    "overhang_angle": 14.04,
    "toggle_louvres_or_fins": False,
    "fins_or_louvers": False,
    "louvre_blade_depth": 0.2,
    "number_louvre_shades": 10,
    "distance_between_louvre_shades": 0.2,
    "louvre_blade_angle": 0.0,
    "louvre_reflectance": 0.35,
    "toggle_blind": False,
    "blind_percentage_cover": 0.0,
    "blind_offset": 0.0,
    "blind_solar_reflectance": 0.5,
    "blind_visible_reflectance": 0.5,
    "blind_diffusive_transmittance": 0.2,
    "blind_specular_transmittance": 0.04,
    "toggle_opaque_blind": False,
    "toggle_perforated_blind": False,
    "blind_perforation_spacing": 0.04,
    "blind_perforation_radius": 0.01,
    "toggle_horizontal_blind": False,
    "toggle_reverse_blind_direction": False,
    "toggle_custom_shading": True,
    "custom_shade_filepath": LINUX_FDIR_JOBS
    / "J7356/Calcs/Modelling/Rhino/Rhino Files/custom_shading_geometry/venetian_blinds/venetian_blinds_-30.json",
    "toggle_external_vertical_fin": False,
    "vertical_fin_height": 2.3,
    "vertical_fin_depth": 0.6,
    "fin_horizontal_offset": 0.5,
    "fin_vertical_offset": 0.7,
    "vertical_fin_reflectance": 0.35,
}

filepath, filename = parameter_file_creation(
    LINUX_FDIR_RHINO_RESULTS, type_name, subtype_name, parameters
)
print(filepath)

Timestep. multiple matches found. using the last one.
['IntText', 'Dropdown']
/home/jovyan/jobs/J7356/Calcs/Modelling/Rhino/Results/horizontal_slats/20231027_093635/external_venetian_blind_rotated/external_venetian_blind_rotated.json


In [34]:
type_name = "vertical_fin"
subtype_name = "optimal_dimensions"

window_height = 1.5
sill_height = 1.1
fin_height = 2.3

fin_start = sill_height + window_height / 2 - fin_height / 2

parameters = {
    "run_irradiance_calc": True,
    "run_df_calc": True,
    "run_climate_based_dl_model": False,
    "epw_filepath": EPW_FPATH,
    "bearing": 180,
    "start_month": 9,
    "start_day": 20,
    "start_hour": 0,
    "end_month": 9,
    "end_day": 20,
    "end_hour": 23,
    "timestep": 1,
    "irr_grid_offset_distance": -0.05,
    "irr_grid_size": 0.1,
    "df_grid_size": 0.1,
    "working_plane_height": 0.8,
    "calc_surface_offset": 0,
    "height_above_ground_level": 0.0,
    "room_width": 3.0,
    "room_depth": 4.0,
    "room_height": 3.0,
    "wall_thickness": 0,
    "slab_thickness": 0,
    "glass_u_value": 0.8,
    "g_value": 0.4,
    "glass_vlt": 0.7,
    "frame_thickness": 0,
    "toggle_window": True,
    "window_offset": 0.0,
    "window_width": 1.0,
    "sill_height": sill_height,
    "window_height": window_height,
    "extra_window_reveal_depth": 0.0,
    "toggle_door": False,
    "door_offset": 0.0,
    "door_width": 1.0,
    "threshold_height": 0.0,
    "door_height": 2.4,
    "toggle_overhang": False,
    "overhang_width": 1.0,
    "overhang_depth": 1.0,
    "overhang_height": 2.6,
    "overhang_offset": 0.0,
    "overhang_reflectance": 0.5,
    "toggle_louvres_or_fins": False,
    "fins_or_louvers": False,
    "louvre_blade_depth": 0.3,
    "number_louvre_shades": 3,
    "distance_between_louvre_shades": 0.2,
    "louvre_blade_angle": 0.0,
    "louvre_reflectance": 0.35,
    "toggle_blind": False,
    "blind_percentage_cover": 0.0,
    "blind_offset": 0.0,
    "blind_solar_reflectance": 0.5,
    "blind_visible_reflectance": 0.5,
    "blind_diffusive_transmittance": 0.2,
    "blind_specular_transmittance": 0.04,
    "toggle_opaque_blind": False,
    "toggle_perforated_blind": False,
    "blind_perforation_spacing": 0.04,
    "blind_perforation_radius": 0.01,
    "toggle_horizontal_blind": False,
    "toggle_reverse_blind_direction": False,
    "toggle_custom_shading": False,
    "custom_shade_filepath": ".",
    "toggle_external_vertical_fin": True,
    "vertical_fin_height": fin_height,
    "vertical_fin_depth": 0.6,
    "fin_horizontal_offset": 0.5,
    "fin_vertical_offset": fin_start,
    "vertical_fin_reflectance": 0.35,
}

filepath, filename = parameter_file_creation(
    LINUX_FDIR_RHINO_RESULTS, type_name, subtype_name, parameters
)
print(filepath)

Timestep. multiple matches found. using the last one.
['IntText', 'Dropdown']
/home/jovyan/jobs/J7356/Calcs/Modelling/Rhino/Results/vertical_fin/20231018_112553/optimal_dimensions/optimal_dimensions.json


In [56]:
type_name = "sliding_shutter"
subtype_names = ["open_louvre", "weather_louvre", "perforated_screen"]

shared_parameters = {
    "run_irradiance_calc": True,
    "run_df_calc": False,
    "run_climate_based_dl_model": True,
    "epw_filepath": EPW_FPATH,
    "bearing": 270,
    "start_month": 9,
    "start_day": 20,
    "start_hour": 0,
    "end_month": 9,
    "end_day": 20,
    "end_hour": 23,
    "timestep": 1,
    "irr_grid_offset_distance": -0.05,
    "irr_grid_size": 0.1,
    "df_grid_size": 0.1,
    "working_plane_height": 0.8,
    "calc_surface_offset": 0.0,
    "height_above_ground_level": 0.0,
    "room_width": 3.0,
    "room_depth": 4.0,
    "room_height": 3.0,
    "wall_thickness": 0.0,
    "slab_thickness": 0.0,
    "glass_u_value": 0.8,
    "g_value": 0.4,
    "glass_vlt": 0.7,
    "frame_thickness": 0.0,
    "toggle_window": True,
    "window_offset": 0.0,
    "window_width": 1.0,
    "sill_height": 1.1,
    "window_height": 1.5,
    "extra_window_reveal_depth": 0.04,
    "toggle_door": False,
    "toggle_overhang": False,
    "toggle_external_vertical_fin": False,
}

open_louvre_params = shared_parameters | {
    "toggle_louvres_or_fins": True,
    "fins_or_louvers": False,
    "louvre_blade_depth": 0.044,
    "number_louvre_shades": 25,
    "distance_between_louvre_shades": 0.07,
    "louvre_blade_angle": 33,
    "louvre_reflectance": 0.35,
    "toggle_blind": False,
    "toggle_custom_shading": False,
}

weather_louvre_params = shared_parameters | {
    "toggle_louvres_or_fins": False,
    "toggle_blind": False,
    "toggle_custom_shading": True,
    "custom_shade_filepath": LINUX_FDIR_JOBS
    / "J7356/Calcs/Modelling/Rhino/Rhino Files/custom_shading_geometry/contrasol_sliding_shutter_40Z/contrasol_40Z.json",
}

perforated_screen_params = shared_parameters | {
    "toggle_louvres_or_fins": False,
    "toggle_blind": True,
    "toggle_custom_shading": False,
    "blind_percentage_cover": 100,
    "blind_offset": 0.04,
    "blind_solar_reflectance": 0.5,
    "blind_visible_reflectance": 0.5,
    "blind_diffusive_transmittance": 0.0,
    "blind_specular_transmittance": 0.15,
    "toggle_opaque_blind": True,
    "toggle_perforated_blind": True,
    "blind_perforation_spacing": 0.04,
    "blind_perforation_radius": 0.01,
    "toggle_horizontal_blind": False,
    "toggle_reverse_blind_direction": False,
}

parameters_list = [open_louvre_params, weather_louvre_params, perforated_screen_params]

timestamp = datetime.datetime.now().strftime("%Y%m%d_%H%M%S")
filepaths = []

for i, name in enumerate(subtype_names):
    fp, fn = fixed_timestamp_parameter_file_creation(
        LINUX_FDIR_RHINO_RESULTS, type_name, name, parameters_list[i], timestamp
    )
    filepaths.append(fp)


print(filepaths)

Timestep. multiple matches found. using the last one.
['IntText', 'Dropdown']
Timestep. multiple matches found. using the last one.
['IntText', 'Dropdown']
Timestep. multiple matches found. using the last one.
['IntText', 'Dropdown']
['/home/jovyan/jobs/J7356/Calcs/Modelling/Rhino/Results/sliding_shutter/20231020_142855/open_louvre/open_louvre.json', '/home/jovyan/jobs/J7356/Calcs/Modelling/Rhino/Results/sliding_shutter/20231020_142855/weather_louvre/weather_louvre.json', '/home/jovyan/jobs/J7356/Calcs/Modelling/Rhino/Results/sliding_shutter/20231020_142855/perforated_screen/perforated_screen.json']


In [60]:
type_name = "overhang"
subtype_names = [
    "unshaded",
    "optimised_overhang",
    "drop_arm_awning",
    "dutch_canopy_awning",
]

shared_parameters = {
    "run_irradiance_calc": True,
    "run_df_calc": True,
    "run_climate_based_dl_model": False,
    "epw_filepath": EPW_FPATH,
    "bearing": 180,
    "start_month": 9,
    "start_day": 20,
    "start_hour": 0,
    "end_month": 9,
    "end_day": 20,
    "end_hour": 23,
    "timestep": 1,
    "irr_grid_offset_distance": -0.05,
    "irr_grid_size": 0.1,
    "df_grid_size": 0.1,
    "working_plane_height": 0.8,
    "calc_surface_offset": 0.0,
    "height_above_ground_level": 0.0,
    "room_width": 3.0,
    "room_depth": 4.0,
    "room_height": 3.0,
    "wall_thickness": 0.0,
    "slab_thickness": 0.0,
    "glass_u_value": 0.8,
    "g_value": 0.4,
    "glass_vlt": 0.7,
    "frame_thickness": 0.0,
    "toggle_window": True,
    "window_offset": 0.0,
    "window_width": 1.0,
    "sill_height": 1.1,
    "window_height": 1.5,
    "extra_window_reveal_depth": 0.0,
    "toggle_door": False,
    "toggle_louvres_or_fins": False,
    "toggle_blind": False,
    "toggle_external_vertical_fin": False,
}

unshaded_params = shared_parameters | {
    "toggle_overhang": False,
    "toggle_custom_shading": False,
}
optimised_overhang_params = shared_parameters | {
    "toggle_overhang": True,
    "overhang_width": 1.5,
    "overhang_depth": 0.75,
    "overhang_height": 2.6,
    "overhang_offset": 0.0,
    "overhang_reflectance": 0.5,
    "overhang_angle": 0,
    "toggle_custom_shading": False,
}

drop_arm_awning_params = shared_parameters | {
    "toggle_overhang": True,
    "overhang_width": 1.4,
    "overhang_depth": 1.03,
    "overhang_height": 2.6,
    "overhang_offset": 0.0,
    "overhang_reflectance": 0.5,
    "overhang_angle": 14.04,
    "toggle_custom_shading": False,
}

dutch_canopy_awning_params = shared_parameters | {
    "toggle_overhang": False,
    "toggle_custom_shading": True,
    "custom_shade_filepath": LINUX_FDIR_JOBS
    / "J7356/Calcs/Modelling/Rhino/Rhino Files/custom_shading_geometry/dutch_canopy/dutch_canopy_awning.json",
}

parameters_list = [
    unshaded_params,
    optimised_overhang_params,
    drop_arm_awning_params,
    dutch_canopy_awning_params,
]

timestamp = datetime.datetime.now().strftime("%Y%m%d_%H%M%S")
filepaths = []

for i, name in enumerate(subtype_names):
    fp, fn = fixed_timestamp_parameter_file_creation(
        LINUX_FDIR_RHINO_RESULTS, type_name, name, parameters_list[i], timestamp
    )
    filepaths.append(fp)


print(filepaths)

Timestep. multiple matches found. using the last one.
['IntText', 'Dropdown']
Timestep. multiple matches found. using the last one.
['IntText', 'Dropdown']
Timestep. multiple matches found. using the last one.
['IntText', 'Dropdown']
Timestep. multiple matches found. using the last one.
['IntText', 'Dropdown']
['/home/jovyan/jobs/J7356/Calcs/Modelling/Rhino/Results/overhang/20231020_170748/unshaded/unshaded.json', '/home/jovyan/jobs/J7356/Calcs/Modelling/Rhino/Results/overhang/20231020_170748/optimised_overhang/optimised_overhang.json', '/home/jovyan/jobs/J7356/Calcs/Modelling/Rhino/Results/overhang/20231020_170748/drop_arm_awning/drop_arm_awning.json', '/home/jovyan/jobs/J7356/Calcs/Modelling/Rhino/Results/overhang/20231020_170748/dutch_canopy_awning/dutch_canopy_awning.json']


In [25]:
type_name = "sliding_shutter"
subtype_name = "weather_louvre"

parameters = {
    "run_irradiance_calc": True,
    "run_df_calc": False,
    "run_climate_based_dl_model": True,
    "epw_filepath": EPW_FPATH,
    "bearing": 270,
    "start_month": 9,
    "start_day": 20,
    "start_hour": 0,
    "end_month": 9,
    "end_day": 20,
    "end_hour": 23,
    "timestep": 1,
    "irr_grid_offset_distance": -0.05,
    "irr_grid_size": 0.1,
    "df_grid_size": 0.1,
    "working_plane_height": 0.8,
    "calc_surface_offset": 0.0,
    "height_above_ground_level": 0.0,
    "room_width": 3.0,
    "room_depth": 4.0,
    "room_height": 3.0,
    "wall_thickness": 0.0,
    "slab_thickness": 0.0,
    "glass_u_value": 0.8,
    "g_value": 0.4,
    "glass_vlt": 0.7,
    "frame_thickness": 0.0,
    "toggle_window": True,
    "window_offset": 0.0,
    "window_width": 1.0,
    "sill_height": 1.1,
    "window_height": 1.5,
    "extra_window_reveal_depth": 0.04,
    "toggle_door": False,
    "door_offset": 0.0,
    "door_width": 1.0,
    "threshold_height": 0.0,
    "door_height": 2.4,
    "toggle_overhang": False,
    "overhang_width": 1.0,
    "overhang_depth": 1.0,
    "overhang_height": 2.6,
    "overhang_offset": 0.0,
    "overhang_reflectance": 0.5,
    "toggle_louvres_or_fins": False,
    "fins_or_louvers": False,
    "louvre_blade_depth": 0.044,
    "number_louvre_shades": 25,
    "distance_between_louvre_shades": 0.07,
    "louvre_blade_angle": 33,
    "louvre_reflectance": 0.35,
    "toggle_blind": False,
    "blind_percentage_cover": 100,
    "blind_offset": 0.04,
    "blind_solar_reflectance": 0.5,
    "blind_visible_reflectance": 0.5,
    "blind_diffusive_transmittance": 0.0,
    "blind_specular_transmittance": 0.15,
    "toggle_custom_shading": True,
    "custom_shade_filepath": LINUX_FDIR_JOBS
    / "J7356/Calcs/Modelling/Rhino/Rhino Files/custom_shading_geometry/contrasol_sliding_shutter_40Z/contrasol_40Z.json",
}

filepath, filename = parameter_file_creation(
    LINUX_FDIR_RHINO_RESULTS, type_name, subtype_name, parameters
)
print(filepath)

Timestep. multiple matches found. using the last one.
['Dropdown', 'IntText']
/home/jovyan/jobs/J7356/Calcs/Modelling/Rhino/Results/sliding_shutter/20231011_102329/weather_louvre/weather_louvre.json


In [15]:
type_name = "sliding_shutter"
subtype_name = "perforated"

parameters = {
    "run_irradiance_calc": True,
    "run_df_calc": False,
    "run_climate_based_dl_model": True,
    "epw_filepath": EPW_FPATH,
    "bearing": 270,
    "start_month": 9,
    "start_day": 20,
    "start_hour": 0,
    "end_month": 9,
    "end_day": 20,
    "end_hour": 23,
    "timestep": 1,
    "irr_grid_offset_distance": -0.05,
    "irr_grid_size": 0.1,
    "df_grid_size": 0.1,
    "working_plane_height": 0.8,
    "calc_surface_offset": 0.0,
    "height_above_ground_level": 0.0,
    "room_width": 3.0,
    "room_depth": 4.0,
    "room_height": 3.0,
    "wall_thickness": 0.0,
    "slab_thickness": 0.0,
    "glass_u_value": 0.8,
    "g_value": 0.4,
    "glass_vlt": 0.7,
    "frame_thickness": 0.0,
    "toggle_window": True,
    "window_offset": 0.0,
    "window_width": 1.0,
    "sill_height": 1.1,
    "window_height": 1.5,
    "extra_window_reveal_depth": 0.04,
    "toggle_door": False,
    "door_offset": 0.0,
    "door_width": 1.0,
    "threshold_height": 0.0,
    "door_height": 2.4,
    "toggle_overhang": False,
    "overhang_width": 1.0,
    "overhang_depth": 1.0,
    "overhang_height": 2.6,
    "overhang_offset": 0.0,
    "overhang_reflectance": 0.5,
    "toggle_louvres_or_fins": False,
    "fins_or_louvers": False,
    "louvre_blade_depth": 0.044,
    "number_louvre_shades": 25,
    "distance_between_louvre_shades": 0.07,
    "louvre_blade_angle": 33,
    "louvre_reflectance": 0.35,
    "toggle_blind": True,
    "blind_percentage_cover": 100,
    "blind_offset": 0.04,
    "blind_solar_reflectance": 0.5,
    "blind_visible_reflectance": 0.5,
    "blind_diffusive_transmittance": 0.0,
    "blind_specular_transmittance": 0.15,
    "toggle_opaque_blind": True,
    "toggle_perforated_blind": True,
    "blind_perforation_spacing": 0.04,
    "blind_perforation_radius": 0.01,
    "toggle_horizontal_blind": False,
    "toggle_reverse_blind_direction": False,
    "toggle_custom_shading": False,
    "custom_shade_filepath": LINUX_FDIR_JOBS
    / "J7356/Calcs/Modelling/Rhino/Rhino Files/custom_shading_geometry/contrasol_sliding_shutter_40Z/contrasol_40Z.json",
}

filepath, filename = parameter_file_creation(
    LINUX_FDIR_RHINO_RESULTS, type_name, subtype_name, parameters
)
print(filepath)

Timestep. multiple matches found. using the last one.
['IntText', 'Dropdown']
/home/jovyan/jobs/J7356/Calcs/Modelling/Rhino/Results/sliding_shutter/20231016_145220/perforated/perforated.json


In [35]:
type_name = "external_blinds"
subtype_name = "0202_White"

parameters = {
    "run_irradiance_calc": True,
    "run_df_calc": False,
    "run_climate_based_dl_model": True,
    "epw_filepath": EPW_FPATH,
    "bearing": 270,
    "start_month": 9,
    "start_day": 20,
    "start_hour": 0,
    "end_month": 9,
    "end_day": 20,
    "end_hour": 23,
    "timestep": 1,
    "irr_grid_offset_distance": -0.05,
    "irr_grid_size": 0.1,
    "df_grid_size": 0.1,
    "working_plane_height": 0.8,
    "calc_surface_offset": 0.0,
    "height_above_ground_level": 0.0,
    "room_width": 3.0,
    "room_depth": 4.0,
    "room_height": 3.0,
    "wall_thickness": 0.0,
    "slab_thickness": 0.0,
    "glass_u_value": 0.8,
    "g_value": 0.4,
    "glass_vlt": 0.7,
    "frame_thickness": 0.0,
    "toggle_window": True,
    "window_offset": 0.0,
    "window_width": 1.0,
    "sill_height": 1.1,
    "window_height": 1.5,
    "extra_window_reveal_depth": 0.0,
    "toggle_door": False,
    "door_offset": 0.0,
    "door_width": 1.0,
    "threshold_height": 0.0,
    "door_height": 2.4,
    "toggle_overhang": False,
    "overhang_width": 1.0,
    "overhang_depth": 1.0,
    "overhang_height": 2.6,
    "overhang_offset": 0.0,
    "overhang_reflectance": 0.5,
    "toggle_louvres_or_fins": False,
    "fins_or_louvers": False,
    "louvre_blade_depth": 0.044,
    "number_louvre_shades": 25,
    "distance_between_louvre_shades": 0.07,
    "louvre_blade_angle": 33,
    "louvre_reflectance": 0.35,
    "toggle_blind": True,
    "blind_percentage_cover": 100,
    "blind_offset": 0.04,
    "blind_solar_reflectance": 0.66,
    "blind_visible_reflectance": 0.73,
    "blind_diffusive_transmittance": 0.16,
    "blind_specular_transmittance": 0.04,
    "toggle_custom_shading": False,
}

filepath, filename = parameter_file_creation(
    LINUX_FDIR_RHINO_RESULTS, type_name, subtype_name, parameters
)
print(filepath)

Timestep. multiple matches found. using the last one.
['IntText', 'Dropdown']
/home/jovyan/jobs/J7356/Calcs/Modelling/Rhino/Results/external_blinds/20231018_132744/0202_White/0202_White.json


In [52]:
type_name = "external_blinds"
subtype_name = "1001_Sable_Grey"

parameters = {
    "run_irradiance_calc": True,
    "run_df_calc": False,
    "run_climate_based_dl_model": True,
    "epw_filepath": EPW_FPATH,
    "bearing": 270,
    "start_month": 9,
    "start_day": 20,
    "start_hour": 0,
    "end_month": 9,
    "end_day": 20,
    "end_hour": 23,
    "timestep": 1,
    "irr_grid_offset_distance": -0.05,
    "irr_grid_size": 0.1,
    "df_grid_size": 0.1,
    "working_plane_height": 0.8,
    "calc_surface_offset": 0.0,
    "height_above_ground_level": 0.0,
    "room_width": 3.0,
    "room_depth": 4.0,
    "room_height": 3.0,
    "wall_thickness": 0.0,
    "slab_thickness": 0.0,
    "glass_u_value": 0.8,
    "g_value": 0.4,
    "glass_vlt": 0.7,
    "frame_thickness": 0.0,
    "toggle_window": True,
    "window_offset": 0.0,
    "window_width": 1.0,
    "sill_height": 1.1,
    "window_height": 1.5,
    "extra_window_reveal_depth": 0.0,
    "toggle_door": False,
    "door_offset": 0.0,
    "door_width": 1.0,
    "threshold_height": 0.0,
    "door_height": 2.4,
    "toggle_overhang": False,
    "overhang_width": 1.0,
    "overhang_depth": 1.0,
    "overhang_height": 2.6,
    "overhang_offset": 0.0,
    "overhang_reflectance": 0.5,
    "toggle_louvres_or_fins": False,
    "fins_or_louvers": False,
    "louvre_blade_depth": 0.044,
    "number_louvre_shades": 25,
    "distance_between_louvre_shades": 0.07,
    "louvre_blade_angle": 33,
    "louvre_reflectance": 0.35,
    "toggle_blind": True,
    "blind_percentage_cover": 100,
    "blind_offset": 0.04,
    "blind_solar_reflectance": 0.28,
    "blind_visible_reflectance": 0.27,
    "blind_diffusive_transmittance": 0.08,
    "blind_specular_transmittance": 0.04,
    "toggle_custom_shading": False,
}

filepath, filename = parameter_file_creation(
    LINUX_FDIR_RHINO_RESULTS, type_name, subtype_name, parameters
)
print(filepath)

Timestep. multiple matches found. using the last one.
['IntText', 'Dropdown']
/home/jovyan/jobs/J7356/Calcs/Modelling/Rhino/Results/external_blinds/20231020_104927/1001_Sable_Grey/1001_Sable_Grey.json


In [53]:
type_name = "external_blinds"
subtype_name = "3030_Charcoal"

parameters = {
    "run_irradiance_calc": True,
    "run_df_calc": False,
    "run_climate_based_dl_model": True,
    "epw_filepath": EPW_FPATH,
    "bearing": 270,
    "start_month": 9,
    "start_day": 20,
    "start_hour": 0,
    "end_month": 9,
    "end_day": 20,
    "end_hour": 23,
    "timestep": 1,
    "irr_grid_offset_distance": -0.05,
    "irr_grid_size": 0.1,
    "df_grid_size": 0.1,
    "working_plane_height": 0.8,
    "calc_surface_offset": 0.0,
    "height_above_ground_level": 0.0,
    "room_width": 3.0,
    "room_depth": 4.0,
    "room_height": 3.0,
    "wall_thickness": 0.0,
    "slab_thickness": 0.0,
    "glass_u_value": 0.8,
    "g_value": 0.4,
    "glass_vlt": 0.7,
    "frame_thickness": 0.0,
    "toggle_window": True,
    "window_offset": 0.0,
    "window_width": 1.0,
    "sill_height": 1.1,
    "window_height": 1.5,
    "extra_window_reveal_depth": 0.0,
    "toggle_door": False,
    "door_offset": 0.0,
    "door_width": 1.0,
    "threshold_height": 0.0,
    "door_height": 2.4,
    "toggle_overhang": False,
    "overhang_width": 1.0,
    "overhang_depth": 1.0,
    "overhang_height": 2.6,
    "overhang_offset": 0.0,
    "overhang_reflectance": 0.5,
    "toggle_louvres_or_fins": False,
    "fins_or_louvers": False,
    "louvre_blade_depth": 0.044,
    "number_louvre_shades": 25,
    "distance_between_louvre_shades": 0.07,
    "louvre_blade_angle": 33,
    "louvre_reflectance": 0.35,
    "toggle_blind": True,
    "blind_percentage_cover": 100,
    "blind_offset": 0.02,
    "blind_solar_reflectance": 0.06,
    "blind_visible_reflectance": 0.06,
    "blind_diffusive_transmittance": 0.00,
    "blind_specular_transmittance": 0.04,
    "toggle_custom_shading": False,
}

filepath, filename = parameter_file_creation(
    LINUX_FDIR_RHINO_RESULTS, type_name, subtype_name, parameters
)
print(filepath)

Timestep. multiple matches found. using the last one.
['IntText', 'Dropdown']
/home/jovyan/jobs/J7356/Calcs/Modelling/Rhino/Results/external_blinds/20231020_132543/3030_Charcoal/3030_Charcoal.json


In [21]:
fpath = "/home/jovyan/jobs/J7356/Calcs/Modelling/Rhino/Results/sliding_shutter/20231009_162857/open_louvre/open_louvre.json"
# display(read_json(fpath))

In [ ]:
results_folder = "/home/jovyan/jobs/J7356/Calcs/Modelling/Rhino/Results"
epw_fp = r"J:\J7356\Calcs\Modelling\Rhino\WeatherFiles\London_LHR_DSY1_2020High50.epw"
type_name = "horizontal_slats"

global_parameters = {
    "run_irradiance_calc": True,
    "run_df_calc": True,
    "run_climate_based_dl_model": False,
    "epw_filepath": epw_fp,
    "bearing": 180,
    "start_month": 5,
    "start_day": 1,
    "start_hour": 0,
    "end_month": 9,
    "end_day": 30,
    "end_hour": 23,
    "timestep": 1,
    "irr_grid_offset_distance": -0.05,
    "irr_grid_size": 0.1,
    "df_grid_size": 0.1,
    "working_plane_height": 0.8,
    "calc_surface_offset": 0.0,
    "height_above_ground_level": 0.0,
    "room_width": 3.0,
    "room_depth": 4.0,
    "room_height": 3.0,
    "wall_thickness": 0.0,
    "slab_thickness": 0.0,
    "glass_u_value": 0.8,
    "g_value": 0.4,
    "glass_vlt": 0.7,
    "frame_thickness": 0.0,
    "toggle_window": True,
    "window_offset": 0.0,
    "window_width": 1.0,
    "sill_height": 1.1,
    "window_height": 1.5,
    "extra_window_reveal_depth": 0,
    "toggle_door": False,
    "door_offset": 0.0,
    "door_width": 1.0,
    "threshold_height": 0.0,
    "door_height": 2.4,
    "toggle_overhang": False,
    "overhang_width": 1.0,
    "overhang_depth": 1.0,
    "overhang_height": 2.6,
    "overhang_offset": 0.0,
    "overhang_reflectance": 0.5,
    "toggle_louvres_or_fins": True,
    "fins_or_louvers": False,
    "louvre_blade_depth": 0.3,
    "number_louvre_shades": 3,
    "distance_between_louvre_shades": 0.2,
    "louvre_blade_angle": 0.0,
    "louvre_reflectance": 0.35,
    "toggle_blind": True,
    "blind_percentage_cover": 100,
    "blind_offset": 0.04,
    "blind_solar_reflectance": 0.5,
    "blind_visible_reflectance": 0.5,
    "blind_diffusive_transmittance": 0.0,
    "blind_specular_transmittance": 0.15,
    "toggle_custom_shading": False
}

blade_depth_arr = np.linspace(0.1, 0.4, 4)
blade_sep_arr = np.linspace(0.05, 0.1, 2)
for i, bd in enumerate(blade_depth_arr):
    for j, bs in enumerate

In [66]:
blade_depth_arr = np.linspace(0.1, 0.4, 4)
blade_sep_arr = np.linspace(0.05, 0.1, 2)

print(blade_sep_arr)

[0.05 0.1 ]


In [25]:
from maplocal import maplocal

In [38]:
WINDOWS_FDIR_JOBS = pathlib.PureWindowsPath("J:\\")

In [41]:
str(
    maplocal(
        path=LINUX_FDIR_RHINO_RESULTS,
        oldroot=LINUX_FDIR_JOBS,
        newroot=WINDOWS_FDIR_JOBS,
    )
).replace("/", "\\")

'J:\\J7356\\Calcs\\Modelling\\Rhino'

In [13]:
FDIR_RHINO_RESULTS = FDIR_RHINO_CALCS / "Results"

In [15]:
folder = "/home/jovyan/jobs/J7356/Calcs/Modelling/Rhino/Results"
epw_fp = "/home/jovyan/jobs/J7356/Calcs/Modelling/Rhino/WeatherFiles/London_LHR_DSY1_2020High50.epw"
parameter_name = "overhang_depth"
start_value = 0
end_value = 2
step = 0.1
global_parameters = {
    "run_irradiance_calc": True,
    "run_df_calc": True,
    "run_climate_based_dl_model": False,
    "epw_filepath": epw_fp,
    "north_angle": 180,
    "start_month": 5,
    "start_day": 1,
    "start_hour": 0,
    "end_month": 9,
    "end_day": 30,
    "end_hour": 23,
    "timestep": 1,
    "irr_grid_offset_distance": -0.05,
    "irr_grid_size": 0.1,
    "df_grid_size": 0.1,
    "working_plane_height": 0.8,
    "calc_surface_offset": 0.25,
    "height_above_ground_level": 0.0,
    "room_width": 3.0,
    "room_depth": 4.0,
    "room_height": 3.0,
    "wall_thickness": 0.0,
    "slab_thickness": 0.4,
    "glass_u_value": 0.8,
    "g_value": 0.4,
    "glass_vlt": 0.7,
    "frame_thickness": 0.0,
    "toggle_window": True,
    "window_offset": 0.0,
    "window_width": 1.0,
    "sill_height": 1.1,
    "window_height": 1.5,
    "toggle_door": False,
    "door_offset": 0.0,
    "door_width": 1.0,
    "threshold_height": 0.0,
    "door_height": 2.4,
    "toggle_overhang": True,
    "overhang_width": 1.0,
    # 'overhang_depth': 1.0,
    "overhang_height": 2.6,
    "overhang_offset": 0.0,
    "overhang_reflectance": 0.5,
    "toggle_louvres_or_fins": False,
    "fins_or_louvers": False,
    "louvre_blade_depth": 0.3,
    "number_louvre_shades": 3,
    "distance_between_louvre_shades": 0.2,
    "louvre_blade_angle": 0.0,
    "louvre_reflectance": 0.35,
    "toggle_blind": False,
    "blind_percentage_cover": 0.0,
    "blind_offset": 0.0,
    "blind_solar_reflectance": 0.5,
    "blind_visible_reflectance": 0.5,
    "blind_diffusive_transmittance": 0.2,
    "blind_specular_transmittance": 0.04,
}

names_list, fn_list, fp_list = parameter_variation(
    folder,
    parameter_name,
    start_value,
    end_value,
    step,
    global_parameters,
    overwrite=True,
)
print(names_list)

Timestep. multiple matches found. using the last one.
['IntText', 'Dropdown']
['overhang_depth_0.0', 'overhang_depth_0.1', 'overhang_depth_0.2', 'overhang_depth_0.3', 'overhang_depth_0.4', 'overhang_depth_0.5', 'overhang_depth_0.6', 'overhang_depth_0.7', 'overhang_depth_0.8', 'overhang_depth_0.9', 'overhang_depth_1.0', 'overhang_depth_1.1', 'overhang_depth_1.2', 'overhang_depth_1.3', 'overhang_depth_1.4', 'overhang_depth_1.5', 'overhang_depth_1.6', 'overhang_depth_1.7', 'overhang_depth_1.8', 'overhang_depth_1.9', 'overhang_depth_2.0']


In [19]:
def read_json(path, filename=""):
    if filename:
        filepath = os.path.join(path, filename)
    else:
        filepath = os.path.join(path)
    f = open(filepath, "r")
    return json.load(f, object_pairs_hook=OrderedDict)

In [10]:
path = "/home/jovyan/jobs/J7356/Calcs/Modelling/Rhino/Results/overhang_depth/20230928_134707/overhang_depth_1.0/20231002_112719/overhang_depth_1.0_results.json"
# path = "jobs/J7356/Calcs/Modelling/Rhino/Results/overhang_depth/20230928_134707/overhang_depth_1.0/20231002_112719/overhang_depth_1.0_results.json"
results = read_json(path)
print(results)

{
  "Peak Solar Gain (W/m^2)": 415.5753,
  "Cumulative Solar Gain (kWh/m^2)": 127.8999,
  "Median DF": 1.077859065,
  "Min DF over 95% of Grid": 0.608047248
}


In [11]:
def read_results(path, filename=""):
    if filename:
        filepath = os.path.join(path, filename)
    else:
        filepath = os.path.join(path)
    f = open(filepath, "r")
    data = f.read()
    return data

In [19]:
data_path = r"/home/jovyan/jobs/J7356/Calcs/Modelling/Rhino/Results/overhang_depth/20230928_134707/overhang_depth_1.0/20231002_112719/irradiance_calc/annual_irradiance/results/total/ParametricModel_ExteriorApertures.ill"
data = read_results(data_path)

In [18]:
import pandas as pd

In [27]:
sun_hours_path = "/home/jovyan/jobs/J7356/Calcs/Modelling/Rhino/Results/overhang_depth/20230928_134707/overhang_depth_1.0/20231002_112719/irradiance_calc/annual_irradiance/results/total/sun-up-hours.txt"

In [40]:
headings = pd.read_csv(sun_hours_path, header=None, index_col=False)
display(headings[0])

0       2885.5
1       2886.5
2       2887.5
3       2888.5
4       2889.5
         ...  
2270    6541.5
2271    6542.5
2272    6543.5
2273    6544.5
2274    6545.5
Name: 0, Length: 2275, dtype: float64

In [41]:
data = pd.read_csv(data_path, sep="\t", header=None, names=headings[0], index_col=False)
display(data)

,2885.5,2886.5,2887.5,2888.5,2889.5,2890.5,2891.5,2892.5,2893.5,2894.5,...,6536.5,6537.5,6538.5,6539.5,6540.5,6541.5,6542.5,6543.5,6544.5,6545.5
0,1.359737,3.047492,6.934735,14.625714,13.141769,16.186256,13.287035,14.811044,14.467670,11.953865,...,5.975844,8.237317,5.427676,9.146648,7.642336,6.094180,5.509249,12.047630,12.917831,1.555444
1,1.893480,3.923753,8.895997,18.644203,16.825993,20.726345,17.041311,19.015511,18.617077,15.412012,...,7.749542,10.701182,7.005220,11.888375,9.907790,7.890609,7.156035,18.213891,22.039692,2.279981
2,2.318789,4.640259,10.494974,21.901480,19.854261,24.491980,20.154976,22.519732,22.086523,18.298914,...,9.204591,12.735155,8.304395,14.173940,11.787734,9.377207,8.523500,22.554967,23.445277,2.861725
3,2.678376,5.379774,12.164541,25.388138,23.093872,28.582434,23.499678,26.319328,25.848387,21.393141,...,10.716683,14.866383,9.655231,16.593229,13.768571,10.933972,9.953199,52.216316,25.731275,3.108008
4,2.951983,5.960445,13.476048,28.114405,25.638306,31.805252,26.153166,29.342932,28.850899,23.865526,...,11.888782,16.512016,10.716563,18.480932,15.327471,12.162445,11.081098,59.012599,31.611884,3.881110
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
121,4.291956,8.233172,19.206003,43.485363,36.991634,45.478710,35.985275,39.865158,38.320637,31.105318,...,17.351589,23.999725,14.573750,25.457827,20.679504,16.231506,14.562430,20.680939,17.840937,3.153613
122,4.323193,8.441412,19.713394,44.809708,38.085278,46.879017,37.014790,41.008953,39.383068,31.923061,...,17.817013,24.668764,14.953040,26.166740,21.229120,16.650331,14.932232,20.168310,18.419289,3.165824
123,4.335739,8.627826,20.164688,45.994106,39.080978,48.174507,37.950718,42.035587,40.328690,32.653267,...,18.248156,25.297676,15.296304,26.824890,21.732458,17.029984,15.268435,20.526577,17.374685,3.152375
124,4.361681,8.758161,20.485767,46.888908,39.838364,49.185444,38.640396,42.790882,41.018120,33.167793,...,18.562874,25.767693,15.539848,27.310003,22.094353,17.298969,15.506218,20.692200,18.442537,3.196344


In [ ]:
from pydantic import Field
from ipyautoui.basemodel import BaseModel


class OverrideIpywidgets(BaseModel):
    """sometimes it isn't possible to guess what widget to use based on type information.
    For example, the Combobox has the same inputs as a Dropdown. You can specify to use
    a specify widget using the `autoui` field.
    """

    combobox: str = Field(
        default=0.8,
        autoui="ipyautoui.autowidgets.Combobox",
    )

In [16]:
import matplotlib as mpl
import matplotlib.pyplot as plt

In [46]:
cmap = "plasma"
colours = []
noOfDiv = 11
colourmaplist = plt.get_cmap(cmap)
for i in range(int(256 / (noOfDiv)) + 1):
    colour = list(colourmaplist(i * noOfDiv))
    colour = [int(round(num * 255, 0)) for num in colour]
    print(str(colour[:-1])[1:-1])
    colours.append(colour)

# print(colours)

13, 8, 135
40, 5, 146
60, 4, 155
78, 2, 162
96, 1, 166
113, 0, 168
129, 4, 167
145, 14, 163
160, 26, 156
173, 39, 147
186, 51, 136
197, 64, 126
207, 76, 116
217, 88, 106
226, 101, 97
233, 114, 87
240, 128, 78
246, 143, 68
250, 158, 59
253, 174, 50
254, 190, 42
252, 208, 37
247, 226, 37
241, 245, 37
